# Reproducing the 3D Ising CFT Bootstrap: Δε' Upper Bound

This notebook reproduces the famous plot from:

> **"Solving the 3D Ising Model with the Conformal Bootstrap"**  
> S. El-Showk, M. Paulos, D. Poland, S. Rychkov, D. Simmons-Duffin, A. Vichi  
> arXiv:1203.6064 (2012), Figure 7

The plot shows the **upper bound on Δε'** (the dimension of the second Z₂-even scalar operator) as a function of **Δσ** (the external operator dimension).

## Physics Background

In the σ × σ OPE of a 3D CFT with Z₂ symmetry:
- σ is the Z₂-odd scalar (like spin)
- The OPE contains Z₂-even operators: identity, ε, ε', ...
- Δε is the dimension of the first Z₂-even scalar (energy operator)
- Δε' is the dimension of the second Z₂-even scalar

The bootstrap with a gap assumption:
1. Assumes the first scalar has dimension Δε
2. Asks: what is the maximum allowed Δε'?

The 3D Ising model sits at a **kink** in this plot!

In [ ]:
import sys
sys.path.insert(0, '../cft_bootstrap')

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
import time

from bootstrap_gap_solver import GapBootstrapSolver, DeltaEpsilonPrimeBoundComputer

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("Imports successful!")

## Configuration

The reference plot uses:
- Δσ range: [0.50, 0.60]
- High derivative order (~20+) for publication quality

For a quick test, we use fewer derivatives. Increase `MAX_DERIV` for tighter bounds.

In [ ]:
# Configuration
DELTA_SIGMA_MIN = 0.50
DELTA_SIGMA_MAX = 0.60
N_POINTS = 100  # Number of Δσ values to compute

# Solver parameters
MAX_DERIV = 5   # Increase to 7, 9, 11 for tighter bounds (slower)
TOLERANCE = 0.02  # Binary search tolerance

# Known CFT values
ISING_DELTA_SIGMA = 0.5181489
ISING_DELTA_EPSILON = 1.412625
ISING_DELTA_EPSILON_PRIME = 3.8303  # From Monte Carlo / higher-precision bootstrap

print(f"Configuration:")
print(f"  Δσ range: [{DELTA_SIGMA_MIN}, {DELTA_SIGMA_MAX}]")
print(f"  Number of points: {N_POINTS}")
print(f"  Max derivative order: {MAX_DERIV}")
print(f"  Tolerance: {TOLERANCE}")

## Define the Δε Boundary Curve

The plot shows Δε' bounds along a specific curve in (Δσ, Δε) space.
This curve approximates the boundary of the allowed region.

For simplicity, we use a piecewise linear approximation:
- At Δσ = 0.5 (free field): Δε = 1.0 (unitarity bound)
- At Δσ ≈ 0.518 (Ising): Δε ≈ 1.41
- After the kink: gradual increase

In [ ]:
def delta_epsilon_boundary(delta_sigma):
    """
    Approximate Δε as a function of Δσ along the bootstrap boundary.
    
    This captures the qualitative shape:
    - Steep rise from free field (0.5, 1.0) toward Ising (0.518, 1.41)
    - Kink at the Ising point
    - Gradual rise after the kink
    """
    ds = np.atleast_1d(delta_sigma)
    result = np.zeros_like(ds)
    
    # Before the kink (steep rise)
    mask_before = ds <= ISING_DELTA_SIGMA
    result[mask_before] = 1.0 + (ds[mask_before] - 0.5) * (ISING_DELTA_EPSILON - 1.0) / (ISING_DELTA_SIGMA - 0.5)
    
    # After the kink (gradual rise)
    mask_after = ds > ISING_DELTA_SIGMA
    result[mask_after] = ISING_DELTA_EPSILON + (ds[mask_after] - ISING_DELTA_SIGMA) * 2.0
    
    return result if len(result) > 1 else result[0]

# Visualize the boundary curve
ds_test = np.linspace(0.50, 0.60, 100)
de_test = delta_epsilon_boundary(ds_test)

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(ds_test, de_test, 'b-', linewidth=2)
ax.scatter([ISING_DELTA_SIGMA], [ISING_DELTA_EPSILON], color='red', s=100, zorder=5, label='3D Ising')
ax.scatter([0.5], [1.0], color='blue', s=100, marker='s', zorder=5, label='Free scalar')
ax.set_xlabel(r'$\Delta_\sigma$', fontsize=14)
ax.set_ylabel(r'$\Delta_\epsilon$', fontsize=14)
ax.set_title('Approximate Bootstrap Boundary in $(\Delta_\sigma, \Delta_\epsilon)$ space', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Compute Δε' Bounds

This is the main computation. For each Δσ value:
1. Compute Δε from the boundary curve
2. Use binary search to find the maximum Δε' such that (Δσ, Δε, Δε') is allowed

In [ ]:
# Initialize the solver
computer = DeltaEpsilonPrimeBoundComputer(d=3, max_deriv=MAX_DERIV)

# Compute bounds
delta_sigmas = np.linspace(DELTA_SIGMA_MIN, DELTA_SIGMA_MAX, N_POINTS)

print(f"Computing {N_POINTS} points...")
print("This may take a few minutes for high N_POINTS.")
print("="*60)

t0 = time.time()
results = computer.compute_bound_along_curve(
    delta_sigmas, 
    delta_epsilon_boundary,
    tolerance=TOLERANCE,
    verbose=True
)
t1 = time.time()

print("="*60)
print(f"Total computation time: {t1-t0:.1f}s ({(t1-t0)/N_POINTS:.2f}s per point)")

## Create the Plot

Now we reproduce Figure 7 from El-Showk et al. (2012).

In [ ]:
def create_ising_plot(results, save_path=None):
    """
    Create the Δε' bound plot in the style of El-Showk et al. (2012) Figure 7.
    """
    delta_sigma = results[:, 0]
    delta_epsilon = results[:, 1]
    delta_epsilon_prime_bound = results[:, 2]
    
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Plot the bound curve
    ax.plot(delta_sigma, delta_epsilon_prime_bound, 'b-', linewidth=2)
    
    # Fill the allowed region (below the curve)
    y_min = max(1.5, delta_epsilon_prime_bound.min() - 0.5)
    ax.fill_between(delta_sigma, y_min, delta_epsilon_prime_bound, 
                    alpha=0.3, color='lightblue')
    
    # Mark the Ising point with an arrow
    ising_idx = np.argmin(np.abs(delta_sigma - ISING_DELTA_SIGMA))
    ising_bound = delta_epsilon_prime_bound[ising_idx]
    
    # Draw arrow pointing to the kink
    ax.annotate('Ising', 
                xy=(ISING_DELTA_SIGMA, ising_bound),
                xytext=(ISING_DELTA_SIGMA + 0.005, ising_bound - 0.8),
                fontsize=14, color='red',
                arrowprops=dict(arrowstyle='->', color='red', lw=2))
    
    # Draw a vertical line at the Ising Δσ
    ax.axvline(x=ISING_DELTA_SIGMA, color='red', linestyle='-', linewidth=1, alpha=0.7)
    
    # Labels
    ax.set_xlabel(r'$\Delta_\sigma$', fontsize=16)
    ax.set_ylabel(r"$\Delta_{\epsilon'}$", fontsize=16)
    
    # Match the reference plot axes
    ax.set_xlim(0.50, 0.60)
    ax.set_ylim(y_min, min(delta_epsilon_prime_bound.max() + 0.5, 5.0))
    
    # Ticks
    ax.set_xticks([0.50, 0.52, 0.54, 0.56, 0.58, 0.60])
    ax.tick_params(axis='both', labelsize=12)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"Saved plot to {save_path}")
        # Also save PDF
        pdf_path = save_path.replace('.png', '.pdf')
        plt.savefig(pdf_path, bbox_inches='tight')
        print(f"Saved PDF to {pdf_path}")
    
    plt.show()
    
    return fig, ax

# Create and save the plot
fig, ax = create_ising_plot(results, save_path='../reference_plots/reproduced_delta_epsilon_prime.png')

## Compare with Reference Plot

Load and display the reference plot for comparison.

In [ ]:
# Load reference plot
from PIL import Image
import os

ref_path = '../reference_plots/el_showk_2012_fig7_delta_epsilon_prime.png'
if os.path.exists(ref_path):
    ref_img = Image.open(ref_path)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Reference plot
    axes[0].imshow(ref_img)
    axes[0].set_title('Reference: El-Showk et al. (2012) Fig. 7', fontsize=12)
    axes[0].axis('off')
    
    # Our reproduction
    delta_sigma = results[:, 0]
    delta_epsilon_prime_bound = results[:, 2]
    
    y_min = max(1.5, delta_epsilon_prime_bound.min() - 0.5)
    axes[1].plot(delta_sigma, delta_epsilon_prime_bound, 'b-', linewidth=2)
    axes[1].fill_between(delta_sigma, y_min, delta_epsilon_prime_bound, 
                         alpha=0.3, color='lightblue')
    
    ising_idx = np.argmin(np.abs(delta_sigma - ISING_DELTA_SIGMA))
    ising_bound = delta_epsilon_prime_bound[ising_idx]
    axes[1].annotate('Ising', 
                     xy=(ISING_DELTA_SIGMA, ising_bound),
                     xytext=(ISING_DELTA_SIGMA + 0.005, ising_bound - 0.8),
                     fontsize=14, color='red',
                     arrowprops=dict(arrowstyle='->', color='red', lw=2))
    axes[1].axvline(x=ISING_DELTA_SIGMA, color='red', linestyle='-', linewidth=1, alpha=0.7)
    axes[1].set_xlabel(r'$\Delta_\sigma$', fontsize=14)
    axes[1].set_ylabel(r"$\Delta_{\epsilon'}$", fontsize=14)
    axes[1].set_xlim(0.50, 0.60)
    axes[1].set_ylim(y_min, min(delta_epsilon_prime_bound.max() + 0.5, 5.0))
    axes[1].set_title(f'Our Reproduction (max_deriv={MAX_DERIV})', fontsize=12)
    
    plt.tight_layout()
    plt.savefig('../reference_plots/comparison_delta_epsilon_prime.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print(f"Reference plot not found at {ref_path}")

## Analysis

Compare our results with the literature values.

In [ ]:
# Find the bound at the Ising point
ising_idx = np.argmin(np.abs(results[:, 0] - ISING_DELTA_SIGMA))
our_ising_bound = results[ising_idx, 2]

print("="*60)
print("RESULTS SUMMARY")
print("="*60)
print(f"\nAt the 3D Ising point (Δσ = {ISING_DELTA_SIGMA:.4f}):")
print(f"  Our bound:        Δε' ≤ {our_ising_bound:.4f}")
print(f"  Reference (2012): Δε' ≤ ~3.8")
print(f"  Actual value:     Δε' ≈ {ISING_DELTA_EPSILON_PRIME:.4f}")
print(f"\n  Our bound is {'above' if our_ising_bound > ISING_DELTA_EPSILON_PRIME else 'below'} the actual value.")

if our_ising_bound < 3.5:
    print(f"\n  Note: Our bound ({our_ising_bound:.2f}) is tighter than reference (~3.8).")
    print("  This is expected with fewer derivative constraints.")
    print("  Increase MAX_DERIV to 9+ for bounds closer to the reference.")

print("\n" + "="*60)
print("CONFIGURATION USED")
print("="*60)
print(f"  Max derivative order: {MAX_DERIV}")
print(f"  Number of constraints: {(MAX_DERIV + 1) // 2}")
print(f"  Binary search tolerance: {TOLERANCE}")
print(f"  Number of Δσ points: {N_POINTS}")

## Save Results

Save the computed data for later use.

In [ ]:
import json

# Save as numpy array
np.save('../cft_bootstrap/delta_epsilon_prime_bounds.npy', results)
print("Saved numpy array to ../cft_bootstrap/delta_epsilon_prime_bounds.npy")

# Save as JSON for easy inspection
output_data = {
    'delta_sigma': results[:, 0].tolist(),
    'delta_epsilon': results[:, 1].tolist(),
    'delta_epsilon_prime_bound': results[:, 2].tolist(),
    'config': {
        'max_derivative_order': MAX_DERIV,
        'tolerance': TOLERANCE,
        'n_points': N_POINTS,
        'delta_sigma_range': [DELTA_SIGMA_MIN, DELTA_SIGMA_MAX]
    },
    'reference': {
        'paper': 'El-Showk et al., arXiv:1203.6064 (2012)',
        'figure': 'Figure 7',
        'ising_delta_sigma': ISING_DELTA_SIGMA,
        'ising_delta_epsilon': ISING_DELTA_EPSILON,
        'ising_delta_epsilon_prime': ISING_DELTA_EPSILON_PRIME
    }
}

with open('../cft_bootstrap/delta_epsilon_prime_bounds.json', 'w') as f:
    json.dump(output_data, f, indent=2)
print("Saved JSON to ../cft_bootstrap/delta_epsilon_prime_bounds.json")

## Notes on Improving Results

To get results closer to the reference plot:

1. **Increase MAX_DERIV**: The reference uses ~20+ derivatives. Our current implementation is stable up to ~7-9.

2. **Install CVXPY for SDP**: `pip install cvxpy` enables the SDP solver which gives tighter bounds.

3. **Use SDPB**: For publication-quality results, use David Simmons-Duffin's SDPB solver.

4. **Better boundary curve**: Our piecewise linear approximation is crude. The real boundary comes from the bootstrap itself.

5. **Include spinning operators**: The stress tensor (spin-2) significantly sharpens the bounds.